In [3]:
from scapy.all import rdpcap, TCP, IP
from pathlib import Path
import pandas as pd


In [4]:
# =============================
# PATH PCAP (LANGSUNG)
# =============================
PCAP_NO_IPS = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian\webbrute\serangan_A.pcap")
PCAP_IPS    = Path(r"D:\BOOSTIFY-IOTML\snort\pengujian_model\ae\webbrute\serangan_pengujian.pcap")

# =============================
# FUNGSI ANALISIS QoS (VALID)
# =============================
def analyze_qos(pcap_path, label):
    packets = rdpcap(str(pcap_path))
    rows = []

    for pkt in packets:
        if pkt.haslayer(TCP) and pkt.haslayer(IP):
            rows.append({
                "time": float(pkt.time),
                "seq": pkt[TCP].seq,
                "length": len(pkt)
            })

    df = pd.DataFrame(rows)

    if df.empty:
        raise ValueError(f"Tidak ada paket TCP di {label}")

    df = df.sort_values("time").reset_index(drop=True)

    # =============================
    # THROUGHPUT (bps)
    # =============================
    duration = df["time"].max() - df["time"].min()
    throughput = (df["length"].sum() * 8) / duration

    # =============================
    # DELAY (INTER-ARRIVAL TIME)
    # =============================
    df["delay"] = df["time"].diff()
    avg_delay = df["delay"].mean()

    # =============================
    # JITTER (VARIASI DELAY)
    # =============================
    df["jitter"] = df["delay"].diff().abs()
    avg_jitter = df["jitter"].mean()

    # =============================
    # PACKET LOSS (TCP SEQ JUMP)
    # =============================
    df["seq_diff"] = df["seq"].diff()
    threshold = df["seq_diff"].median() * 2
    packet_loss_pct = len(df[df["seq_diff"] > threshold]) / len(df) * 100

    return {
        "scenario": label,
        "duration_s": duration,
        "throughput_bps": throughput,
        "avg_delay_s": avg_delay,
        "avg_jitter_s": avg_jitter,
        "packet_loss_pct": packet_loss_pct
    }
    

In [5]:
# =============================
# ANALISIS SEBELUM & SESUDAH IPS
# =============================
res_no_ips = analyze_qos(PCAP_NO_IPS, "No IPS")
res_ips    = analyze_qos(PCAP_IPS, "With IPS")

df_result = pd.DataFrame([res_no_ips, res_ips])
print("\n=== HASIL QoS ===")
print(df_result)


=== HASIL QoS ===
   scenario  duration_s  throughput_bps  avg_delay_s  avg_jitter_s  \
0    No IPS  163.247056    15408.204359     0.138462      0.239505   
1  With IPS  158.047174    12500.369037     0.220121      0.420121   

   packet_loss_pct  
0        45.508475  
1        36.717663  


In [6]:
# =============================
# PERBANDINGAN QoS
# =============================
comparison = pd.DataFrame({
    "metric": [
        "throughput_bps",
        "avg_delay_s",
        "avg_jitter_s",
        "packet_loss_pct"
    ],
    "no_ips": [
        res_no_ips["throughput_bps"],
        res_no_ips["avg_delay_s"],
        res_no_ips["avg_jitter_s"],
        res_no_ips["packet_loss_pct"]
    ],
    "with_ips": [
        res_ips["throughput_bps"],
        res_ips["avg_delay_s"],
        res_ips["avg_jitter_s"],
        res_ips["packet_loss_pct"]
    ]
})

comparison["delta"] = comparison["with_ips"] - comparison["no_ips"]
comparison["delta_percent"] = (comparison["delta"] / comparison["no_ips"]) * 100

print("\n=== PERBANDINGAN QoS ===")
print(comparison)


=== PERBANDINGAN QoS ===
            metric        no_ips      with_ips        delta  delta_percent
0   throughput_bps  15408.204359  12500.369037 -2907.835322     -18.871993
1      avg_delay_s      0.138462      0.220121     0.081659      58.975697
2     avg_jitter_s      0.239505      0.420121     0.180617      75.412625
3  packet_loss_pct     45.508475     36.717663    -8.790811     -19.316866
